In [7]:
# train_vgg16_ecg_improved.py

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast

# 1) Device & mixed-precision scaler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = GradScaler()

# 2) Hyperparameters
DATA_DIR      = r"D:\res_work\ECG_analysis_for_CVD\PCA\extrct"
NUM_CLASSES   = 4
BATCH_SIZE    = 4       # small per-GPU batch for VRAM headroom
ACCUM_STEPS   = 8       # effective batch = 4 × 8 = 32
TOTAL_EPOCHS  = 30
LR_HEAD       = 1e-3    # head learning rate
LR_FEAT       = 1e-4    # Conv4+5 learning rate
WEIGHT_DECAY  = 1e-4
TRAIN_RATIO   = 0.7
TEST_RATIO    = 0.2
VAL_RATIO     = 0.1

# 3) Transforms
train_tf = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandomAffine(degrees=15, translate=(0.1,0.1), shear=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.02,0.15), ratio=(0.3,3.3)),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

# 4) Load dataset & split by index
full_ds = datasets.ImageFolder(DATA_DIR)
N = len(full_ds)
n_train = int(TRAIN_RATIO * N)
n_test  = int(TEST_RATIO  * N)
n_val   = N - n_train - n_test

g = torch.Generator().manual_seed(42)
perm = torch.randperm(N, generator=g).tolist()
train_idx = perm[:n_train]
test_idx  = perm[n_train:n_train+n_test]
val_idx   = perm[n_train+n_test:]

# 5) Compute per-sample weights for Balanced Sampling
train_targets = [full_ds.targets[i] for i in train_idx]
class_counts  = np.bincount(train_targets, minlength=NUM_CLASSES)
class_weights = 1. / class_counts
sample_weights = [class_weights[t] for t in train_targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# 6) Create Subsets with transforms
train_ds = Subset(datasets.ImageFolder(DATA_DIR, transform=train_tf), train_idx)
val_ds   = Subset(datasets.ImageFolder(DATA_DIR, transform=val_tf),   val_idx)
test_ds  = Subset(datasets.ImageFolder(DATA_DIR, transform=val_tf),   test_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          sampler=sampler, num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4)

# 7) Model setup: VGG-16, replace head, unfreeze Conv4+Conv5
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
# Freeze first three conv blocks (layers 0‐15)
for p in model.features[:16].parameters():
    p.requires_grad = False
# Unfreeze Conv4 & Conv5 (layers 16 on)
for p in model.features[16:].parameters():
    p.requires_grad = True

# Replace classifier
in_f = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_f, NUM_CLASSES)
model = model.to(device)

# 8) Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None):
        super().__init__()
        self.gamma = gamma
        self.ce    = nn.CrossEntropyLoss(weight=weight)
    def forward(self, logits, targets):
        logp = -self.ce(logits, targets)
        p    = torch.exp(logp)
        return -((1 - p) ** self.gamma) * logp

criterion = FocalLoss(gamma=2.0)

# 9) Optimizer & Cosine Annealing LR
# Two param-groups: conv features + head
opt = optim.AdamW([
    {'params': model.features[16:].parameters(), 'lr': LR_FEAT},
    {'params': model.classifier.parameters(),    'lr': LR_HEAD}
], weight_decay=WEIGHT_DECAY)

sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=TOTAL_EPOCHS, eta_min=1e-6)

# 10) Training loop with mixed precision & grad accumulation
best_val_acc = 0.0

for epoch in range(1, TOTAL_EPOCHS + 1):
    # — Train —
    model.train()
    running_loss = 0.0
    running_corr = 0
    opt.zero_grad()

    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(device), lbls.to(device)
        with autocast():
            logits = model(imgs)
            loss   = criterion(logits, lbls) / ACCUM_STEPS
        scaler.scale(loss).backward()

        if (i + 1) % ACCUM_STEPS == 0:
            scaler.step(opt)
            scaler.update()
            opt.zero_grad()

        running_loss += loss.item() * ACCUM_STEPS
        running_corr += (logits.argmax(1) == lbls).sum().item()

    train_loss = running_loss / n_train
    train_acc  = running_corr / n_train * 100

    # — Validate —
    model.eval()
    val_loss = 0.0
    val_corr = 0
    with torch.no_grad(), autocast():
        for imgs, lbls in val_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            logits = model(imgs)
            val_loss += criterion(logits, lbls).item()
            val_corr += (logits.argmax(1) == lbls).sum().item()

    val_loss = val_loss / n_val
    val_acc  = val_corr / n_val * 100

    sched.step()

    print(f"Epoch {epoch}/{TOTAL_EPOCHS} | "
          f"Train: loss={train_loss:.4f}, acc={train_acc:.2f}% | "
          f" Val: loss={val_loss:.4f}, acc={val_acc:.2f}%")

    # Save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_vgg16_ecg.pth")
        print("  → New best model saved!")

print(f"\nTraining complete. Best validation accuracy: {best_val_acc:.2f}%")

C:\Users\FireFly\AppData\Local\Temp\ipykernel_11676\3147357983.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\FireFly\AppData\Local\Temp\ipykernel_11676\3147357983.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\FireFly\AppData\Local\Temp\ipykernel_11676\3147357983.py:146: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


Epoch 1/30 | Train: loss=0.2677, acc=27.85% |  Val: loss=0.1815, acc=28.72%
  → New best model saved!
Epoch 2/30 | Train: loss=0.1552, acc=43.23% |  Val: loss=0.1105, acc=60.64%
  → New best model saved!
Epoch 3/30 | Train: loss=0.1190, acc=58.31% |  Val: loss=0.3281, acc=47.87%
Epoch 4/30 | Train: loss=0.1040, acc=62.00% |  Val: loss=0.0428, acc=82.98%
  → New best model saved!
Epoch 5/30 | Train: loss=0.0865, acc=66.92% |  Val: loss=0.0912, acc=62.77%
Epoch 6/30 | Train: loss=0.0800, acc=66.46% |  Val: loss=0.0813, acc=77.66%
Epoch 7/30 | Train: loss=0.0710, acc=70.46% |  Val: loss=0.0542, acc=82.98%
Epoch 8/30 | Train: loss=0.0654, acc=73.08% |  Val: loss=0.0823, acc=75.53%
Epoch 9/30 | Train: loss=0.0638, acc=68.31% |  Val: loss=0.0586, acc=79.79%
Epoch 10/30 | Train: loss=0.0362, acc=78.46% |  Val: loss=0.0609, acc=88.30%
  → New best model saved!
Epoch 11/30 | Train: loss=0.0337, acc=80.46% |  Val: loss=0.0756, acc=82.98%
Epoch 12/30 | Train: loss=0.0287, acc=81.08% |  Val: loss=

In [1]:
import torch
print(torch.__version__)            # Should show something like 2.8.0.dev20250429+cu118
print(torch.cuda.is_available())    # Should be True
print(torch.cuda.get_device_name(0))# Should report your RTX 3050 Ti

2.8.0.dev20250429+cu118
True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [2]:
# infer_vgg16_ecg.py

import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image

# -------------------------------
# 1. Configuration
# -------------------------------
class_names = [
    "Myocardial Infarction",
    "Abnormal Heartbeat",
    "History of MI",
    "Normal"
]

checkpoint_path = r"D:\res_work\ECG_analysis_for_CVD\best_vgg16_ecg.pth"
num_classes     = len(class_names)

# -------------------------------
# 2. Transforms (same as training)
# -------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

# -------------------------------
# 3. Load Model
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
in_features = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(in_features, num_classes)

vgg.load_state_dict(torch.load(checkpoint_path, map_location=device))
vgg = vgg.to(device)
vgg.eval()

# -------------------------------
# 4. Inference Function
# -------------------------------
def classify_ecg(image_path: str) -> str:
    img = Image.open(image_path).convert("RGB")
    inp = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = vgg(inp)
        _, pred = torch.max(outputs, 1)

    return class_names[pred.item()]

# -------------------------------
# 5. Run from CLI
# -------------------------------
if __name__ == "__main__":
    path = input("Enter path to ECG image: ").strip()
    try:
        label = classify_ecg(path)
        print(f"\nPredicted class: {label}")
    except Exception as e:
        print(f"Error: {e}")


Enter path to ECG image:  C:\Users\FireFly\Desktop\MI_100.jpg



Predicted class: Myocardial Infarction


In [ ]:
# train_vgg16_ecg_improved.py

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast

# 1) Device & mixed-precision scaler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = GradScaler()

# 2) Hyperparameters
DATA_DIR      = r"D:\res_work\ECG_analysis_for_CVD\processed_images"
NUM_CLASSES   = 4
BATCH_SIZE    = 4       # small per-GPU batch for VRAM headroom
ACCUM_STEPS   = 8       # effective batch = 4 × 8 = 32
TOTAL_EPOCHS  = 30
LR_HEAD       = 1e-3    # head learning rate
LR_FEAT       = 1e-4    # Conv4+5 learning rate
WEIGHT_DECAY  = 1e-4
TRAIN_RATIO   = 0.7
TEST_RATIO    = 0.2
VAL_RATIO     = 0.1

# 3) Transforms
train_tf = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.RandomAffine(degrees=15, translate=(0.1,0.1), shear=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.02,0.15), ratio=(0.3,3.3)),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

# 4) Load dataset & split by index
full_ds = datasets.ImageFolder(DATA_DIR)
N = len(full_ds)
n_train = int(TRAIN_RATIO * N)
n_test  = int(TEST_RATIO  * N)
n_val   = N - n_train - n_test

g = torch.Generator().manual_seed(42)
perm = torch.randperm(N, generator=g).tolist()
train_idx = perm[:n_train]
test_idx  = perm[n_train:n_train+n_test]
val_idx   = perm[n_train+n_test:]

# 5) Compute per-sample weights for Balanced Sampling
train_targets = [full_ds.targets[i] for i in train_idx]
class_counts  = np.bincount(train_targets, minlength=NUM_CLASSES)
class_weights = 1. / class_counts
sample_weights = [class_weights[t] for t in train_targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# 6) Create Subsets with transforms
train_ds = Subset(datasets.ImageFolder(DATA_DIR, transform=train_tf), train_idx)
val_ds   = Subset(datasets.ImageFolder(DATA_DIR, transform=val_tf),   val_idx)
test_ds  = Subset(datasets.ImageFolder(DATA_DIR, transform=val_tf),   test_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          sampler=sampler, num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4)

# 7) Model setup: VGG-16, replace head, unfreeze Conv4+Conv5
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
# Freeze first three conv blocks (layers 0‐15)
for p in model.features[:16].parameters():
    p.requires_grad = False
# Unfreeze Conv4 & Conv5 (layers 16 on)
for p in model.features[16:].parameters():
    p.requires_grad = True

# Replace classifier
in_f = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_f, NUM_CLASSES)
model = model.to(device)

# 8) Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None):
        super().__init__()
        self.gamma = gamma
        self.ce    = nn.CrossEntropyLoss(weight=weight)
    def forward(self, logits, targets):
        logp = -self.ce(logits, targets)
        p    = torch.exp(logp)
        return -((1 - p) ** self.gamma) * logp

criterion = FocalLoss(gamma=2.0)

# 9) Optimizer & Cosine Annealing LR
# Two param-groups: conv features + head
opt = optim.AdamW([
    {'params': model.features[16:].parameters(), 'lr': LR_FEAT},
    {'params': model.classifier.parameters(),    'lr': LR_HEAD}
], weight_decay=WEIGHT_DECAY)

sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=TOTAL_EPOCHS, eta_min=1e-6)

# 10) Training loop with mixed precision & grad accumulation
best_val_acc = 0.0

for epoch in range(1, TOTAL_EPOCHS + 1):
    # — Train —
    model.train()
    running_loss = 0.0
    running_corr = 0
    opt.zero_grad()

    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(device), lbls.to(device)
        with autocast():
            logits = model(imgs)
            loss   = criterion(logits, lbls) / ACCUM_STEPS
        scaler.scale(loss).backward()

        if (i + 1) % ACCUM_STEPS == 0:
            scaler.step(opt)
            scaler.update()
            opt.zero_grad()

        running_loss += loss.item() * ACCUM_STEPS
        running_corr += (logits.argmax(1) == lbls).sum().item()

    train_loss = running_loss / n_train
    train_acc  = running_corr / n_train * 100

    # — Validate —
    model.eval()
    val_loss = 0.0
    val_corr = 0
    with torch.no_grad(), autocast():
        for imgs, lbls in val_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            logits = model(imgs)
            val_loss += criterion(logits, lbls).item()
            val_corr += (logits.argmax(1) == lbls).sum().item()

    val_loss = val_loss / n_val
    val_acc  = val_corr / n_val * 100

    sched.step()

    print(f"Epoch {epoch}/{TOTAL_EPOCHS} | "
          f"Train: loss={train_loss:.4f}, acc={train_acc:.2f}% | "
          f" Val: loss={val_loss:.4f}, acc={val_acc:.2f}%")

    # Save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_vgg16_(processed)ecg.pth")
        print("  → New best model saved!")

print(f"\nTraining complete. Best validation accuracy: {best_val_acc:.2f}%")

C:\Users\FireFly\AppData\Local\Temp\ipykernel_18852\1246743983.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\FireFly\AppData\Local\Temp\ipykernel_18852\1246743983.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\FireFly\AppData\Local\Temp\ipykernel_18852\1246743983.py:136: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Scalar.cpp:23.)
  running_loss += loss.item() * ACCUM_STEPS
C:\Users\FireFly\AppData\Local\Temp\ipykernel_18852\1246743983.py:146: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with 

Epoch 1/30 | Train: loss=0.2067, acc=37.90% |  Val: loss=0.1423, acc=44.68%
  → New best model saved!
